In [2]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 8.2 MB/s 


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [4]:
(train_images,train_labels),(test_images,test_lables)=keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):
  model=keras.Sequential([
  keras.layers.Conv2D(
      filters=hp.Int("conv_1_filter",min_value=32,max_value=128,step=16),
      kernel_size=hp.Choice("conv_1_kernel",values=[3,5]),
      activation="relu",
      input_shape=(28,28,1),
  ),
  keras.layers.Conv2D(
      filters=hp.Int("conv_2_filter",min_value=32,max_value=64,step=16),
      kernel_size=hp.Choice("conv_2_kernel",values=[3,5]),
      activation="relu",
  ),
  keras.layers.Flatten(),
  keras.layers.Dense(
    units=hp.Int("dense_1_units",min_value=32,max_value=128,step=16),
    activation="relu",
  ),
  keras.layers.Dense(10,activation="softmax")              
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate",values=[1e-2,1e-3])),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  return model


In [9]:
tuner_search=RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory="output",project_name="Mnist Fashion")


In [10]:
tuner_search.search(train_images,train_labels,epochs=5,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.9773333072662354

Best val_accuracy So Far: 0.9904999732971191
Total elapsed time: 00h 04m 45s
INFO:tensorflow:Oracle triggered exit


In [11]:
model=tuner_search.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        76864     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 96)                2973792   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 3,052,106
Trainable params: 3,052,106
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0103 - accuracy: 0.9967 - val_loss: 0.0432 - val_accuracy: 0.9913
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0086 - accuracy: 0.9972 - val_loss: 0.0535 - val_accuracy: 0.9888
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0058 - accuracy: 0.9981 - val_loss: 0.0542 - val_accuracy: 0.9912
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0054 - accuracy: 0.9985 - val_loss: 0.0784 - val_accuracy: 0.9875
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0061 - accuracy: 0.9984 - val_loss: 0.0617 - val_accuracy: 0.9898
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0040 - accuracy: 0.9986 - val_loss: 0.0640 - val_accuracy: 0.9893
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0037 - accuracy: 0.9988 - val_loss: 0.0794 - val_accuracy